In [6]:
from pylabnet.utils.logging.logger import LogClient

from pylabnet.hardware.awg.zi_hdawg import Driver, Sequence, AWGModule

from pylabnet.utils.zi_hdawg_pulseblock_handler.zi_hdawg_pb_handler import DIOPulseBlockHandler

import pylabnet.utils.pulseblock.pulse as po
import pylabnet.utils.pulseblock.pulse_block as pb
from pylabnet.utils.pulseblock.pb_iplot import iplot
from pylabnet.utils.pulseblock.pb_sample import pb_sample
from pylabnet.utils.pulsed_experiments.pulsed_experiment import PulsedExperiment


# Digital pulse playback using `pulseblock` and the DIO output of a ZI HDWAG

In [2]:
dev_id = 'dev8227'

# Instantiate logger.
logger = LogClient(
    host='140.247.189.82',
    port=37542,
    module_tag=f'ZI HDAWG {dev_id}'
)

# Instanciate HDAWG driver.
hd = Driver(dev_id, logger=logger)

Let's start with defining the digital part of a typical rabi-sequence. We use the `rabi_element` function as defined in the `pulseblock` demo notebook:

In [3]:
def rabi_element(tau=0, aom_offset=0):
    rabi_element = pb.PulseBlock(
        p_obj_list=[
            po.PTrue(ch='aom', dur=1.1e-6),
            po.PTrue(ch='ctr', t0=0.5e-6, dur=0.5e-6)
        ]
    )
    temp_t = rabi_element.dur

    rabi_element.insert(
        p_obj=po.PTrue(ch='mw_gate', dur=tau, t0=temp_t+0.7e-6)
    )
    temp_t = rabi_element.dur

    rabi_element.insert(
        p_obj=po.PTrue(ch='aom', t0=temp_t+aom_offset, dur=2e-6)
    )
    rabi_element.insert(
        p_obj=po.PTrue(ch='ctr', t0=temp_t, dur=0.5e-6)
    )

    return rabi_element

In [4]:
# Let's choose a microwave duration of 1us.
tau = 1e-6

rabi_pulse = rabi_element(tau)

In [5]:
# Let's plot it.
iplot(rabi_pulse)

Now, let's turn this `pulseblock` instance into an intruction set for the DIO outputs of the HDAWG, using the `PulsedExperiment` class. We'll first use a standard sequence stored in the `pulsed_experiment/sequence_templates` folder under `base_dig_pulse.seqct`:

```c
while (1) {
                $dig_sequence0$ 
                wait(1000);
}
```

If the optional keyword argument The syntax of the .seqc template files (.seqct files) is the follows: Statements to be replaces by digital pulse-sequences are designated b `$dig_sequencei$`, where i is an integer starting to count from 0. The keyword argument `pulseblocks` of `PulsedExperiment` can be a list of `PulseBlocks`, and the ith pulseblock will replace the ith digital pulse-sequence placeholder `$dig_sequencei$. The delimeter character $ as well as the replacement term `dig_sequence` can be changed by overwriting the appropriate default keword argument during the `PulsedExperiment` instanciation. Now, the ZIHDAWG can be configured to run the sequence in two lines of code: 

In [11]:
# Specify the DIO outputs which drive the MW, counter and aom.
assignment_dict = {
    'mw_gate':   15,
    'ctr':      16,
    'aom':      17,
}

# Which awg core to use
awg_num = 1

In [12]:
# Instanciate pulsed experiment
pe_rabi = PulsedExperiment(
    pulseblocks=rabi_pulse, 
    assignment_dict=assignment_dict, 
    hd=hd, 
    )

# Compile sequence, upload to HDAWG and prepare DIO output
awg = pe_rabi.get_ready(awg_num)

In [10]:
# Now, we can start the sequence playback easily:
awg.start()

It it also possible to upload a sequecen defined at runtime, and to replace additional variables in the sequence template. Additional variables to be replaced must be encapsulated by the 

In [12]:
# This sequence will send out a trigger and execute the commands  dig_pulse (yet to be specified).
sequence_txt = """\

        while (1) {
            if (getUserReg($trigger_user_reg$) == $trigger_up_val$) {
                repeat ($repetitions$) {
                $dig_sequence0$  
                wait(1000);
                }
                setUserReg($trigger_user_reg$, $trigger_down_val$);
                $dig_sequence1$
            }
        }
        """

replacement_dict = {
    "trigger_user_reg": 0,
    "trigger_up_val" : 0,
    "trigger_down_val" : 1
}

In [13]:
# This sequence will send out a trigger and execute the commands  dig_pulse (yet to be specified).
sequence_txt = """\

        while (1) {
            if (getUserReg($trigger_user_reg$) == $trigger_up_val$) {
                repeat ($repetitions$) {
                $dig_sequence0$  
                wait(1000);
                }
                setUserReg($trigger_user_reg$, $trigger_down_val$);
            }
        }
        """

# Create Sequence instance.
seq = Sequence(hd, sequence_txt, ['dig_pulse'])

# Replace dig_pulse placeholder by digital pulse sequence instruction set.
seq.replace_placeholder('dig_pulse', dig_pulse_sequence)

In [14]:
# Create an instance of the AWG Module.
awg = AWGModule(hd, 0)
awg.set_sampling_rate('2.4 GHz') # Set 2.4 GHz sampling rate.

# Upload sequence.
if awg is not None:
    awg.compile_upload_sequence(seq)

AttributeError: 'LogHandler' object has no attribute 'warning'

In [13]:
# Now we're almost ready, we only have to make sure that the 8 bit buses for bits 15, 17, and 31 are driven. 
# This can be done automatically by calling the following:

pb_handler.setup_hd()

In [14]:
# Start the AWG
awg.start()

The output waveform can be verified on the scope, we have successfully reproduced our digital rabi pulse sequence:

![alt text](pictures/pulse_sequence.jpg "Title")


In [15]:
# Let's compare to the target pulse sequence.
iplot(rabi_pulse)